In [1]:
import pandas as pd
import numpy as np
import nltk
from cogroo4py.cogroo import Cogroo
# nltk.download('punkt')
import language_tool_python as lt
import uol_redacoes_xml as uol

### Analysing Possible Erros in Essays

In [2]:
tool = lt.LanguageTool('pt-BR')
cogroo = Cogroo()
essays = uol.load()

In [32]:
def get_essays_corrections_tool(essays, tool):
    data = []
    for i, ess in enumerate(essays):
        sentences = ess.text.replace('\n', '').split('.')
        for sent in sentences:
            matches = tool.check(sent)
            for mat in matches:
                data.append({
                    'essay': i,
                    'error': mat.ruleId,
                    'message': mat.message,
                    'category': mat.category,
                    'frase': sent
                })
    erros_df = pd.DataFrame(data)
    return erros_df

In [37]:
def get_essays_corrections_cogroo(essays, cogroo):
    data = []
    for i, ess in enumerate(essays):
        sentences = ess.text.replace('\n', '').split('.')
        for sent in sentences:
            try:
                doc = cogroo.grammar_check(sent)
                matches = doc.mistakes
                for mat in matches:
                    data.append({
                        'essay': i,
                        'error': mat.rule_id,
                        'message':  mat.short_msg,
                        'frase': sent
                    })
            except:
                print('Missed sentence', sent)
    erros_df = pd.DataFrame(data)
    return erros_df

In [ ]:
tool_df = get_essays_corrections_tool(essays, tool)
tool_df.to_csv('../data/correction_language_tool.csv', sep='|', index=False)
tool_df.head()

In [42]:
cogroo_df = get_essays_corrections_cogroo(essays, cogroo)
cogroo_df.to_csv('../data/correction_cogroo.csv', sep='|', index=False)
cogroo_df.head()

java.lang.NullPointerException: java.lang.NullPointerException: Cannot invoke "String.length()" because "replacement" is null